In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout


In [3]:
#initializing cnn
classifier = Sequential()
classifier.add(Convolution2D(32,(3,3),input_shape=(64,64,1),activation='relu',padding='same'))
classifier.add(Convolution2D(32,(3,3),input_shape=(64,64,1),activation='relu',padding='same'))
classifier.add(MaxPooling2D(pool_size=(3,3),strides=2))
classifier.add(Dropout(0.5))
classifier.add(Convolution2D(64,(3,3),input_shape=(64,64,1),activation='relu',padding='same'))
classifier.add(Convolution2D(64,(3,3),input_shape=(64,64,1),activation='relu',padding='same'))
classifier.add(MaxPooling2D(pool_size=(3,3),strides=2))
classifier.add(Flatten())
classifier.add(Dense(units=64,activation='relu'))
classifier.add(Dense(units=8,activation='softmax'))


In [4]:
classifier.summary()

In [5]:
classifier.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics = ['accuracy'])


In [8]:
from keras.preprocessing.image import ImageDataGenerator

#radom scaling are applied to the images before training the model
#image augmentation
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
samplewise_center=True,
vertical_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        "../input/handgesture/data/train",
        target_size=(64,64),
        batch_size=64,
        color_mode='grayscale',
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        "../input/handgesture/data/test",
        target_size=(64,64),
        batch_size=32,
        color_mode='grayscale',
        class_mode='categorical')

classifier.fit_generator(training_set,
        steps_per_epoch=125,
        epochs=50,
        validation_data=test_set,
        validation_steps=125)

In [9]:

import numpy as np
from keras.preprocessing import image
test_img= image.load_img("../input/handgesture/data/test/fist/10.png",target_size=(64,64),color_mode='grayscale')
test_img=image.img_to_array(test_img)
test_img=np.expand_dims(test_img,axis=0)
result= classifier.predict(test_img)
result


In [10]:
from keras.models import load_model
classifier.save('hand_gestures_1.h5')